# "Covid19 exploration"
> "An EDA of covid19 data using the UK governments python API and Altair for graphics"

- toc: true
- branch: master
- badges: true
- comments: false
- author: Ifan Johnston
- categories: [covid, eda]

{% include info.html text="Note: If you just would like to see the graphs, just use the link in the table of contents to go to the last sections!" %}


# UK Covid19 API

In this post we will explore the data found in the UK governments Covid API (the Python version), which can be found on their [website](https://coronavirus.data.gov.uk/details/developers-guide). Lets import the Python module first and checkout the main function.

In [1]:
from uk_covid19 import Cov19API

import pandas as pd
import altair as alt
import numpy as np

Collapse the following output to see the help documents for `Cov19API`. It tells us that it is a class with the parameters `filters` (a list of `strings`), `structure` (a dictionary with a `str` key and `dict` or `str` value) and `latest_by` (a `str` or `None`). The class also has a method called `get_dataframe()` which will return the data as a Pandas DataFrame.

In [2]:
# collapse_output
help(Cov19API)

Help on class Cov19API in module uk_covid19.api_interface:

class Cov19API(builtins.object)
 |  Cov19API(filters: Iterable[str], structure: Dict[str, Union[dict, str]], latest_by: Union[str, NoneType] = None)
 |  
 |  Interface to access the API service for COVID-19 data in the United Kingdom.
 |  
 |  Parameters
 |  ----------
 |  filters: Iterable[str]
 |      API filters. See the API documentations for additional
 |      information.
 |  
 |  structure: Dict[str, Union[dict, str]]
 |      Structure parameter. See the API documentations for
 |      additional information.
 |  
 |  latest_by: Union[str, None]
 |      Retrieves the latest value for a specific metric. [Default: ``None``]
 |  
 |  Methods defined here:
 |  
 |  __init__(self, filters: Iterable[str], structure: Dict[str, Union[dict, str]], latest_by: Union[str, NoneType] = None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__ = __str__(self)
 |  
 |  __str__(self)
 |      Return st

So now we need to define two things: the `filters` and the `structure`. 

## Data Filters

The filter tells the API what kind of area we would like data about. Valid values for the filters are:

<details class="govuk-details" data-module="govuk-details" open=""><summary class="govuk-details__summary"><span class="govuk-details__summary-text">List of valid <code class="sc-fzoiQi ozSmQ">filters</code></span></summary><div class="govuk-details__text"><dl><dt><code class="sc-fzoiQi ozSmQ">areaType</code></dt><dd>Area type as string</dd><dt><code class="sc-fzoiQi ozSmQ">areaName</code></dt><dd>Area name as string</dd><dt><code class="sc-fzoiQi ozSmQ">areaCode</code></dt><dd>Area Code as string</dd><dt><code class="sc-fzoiQi ozSmQ">date</code></dt><dd>Date as string [<code class="sc-fzoiQi ozSmQ">YYYY-MM-DD</code>]</dd></dl></div></details>

We must specify the `areaType`, so we will set it to `nation`. This will give us the data on the country level - so the total data for Wales, Scotland, Northen Ireland and England.

In [3]:
filter_all_nations = [
    "areaType=nation"
]

In [4]:
filter_all_uk = [
    "areaType=overview"
]

Other options for `areaType` will give:

- `overview` overview data for the UK
- `region` Region data (regions for England only)
- `nhsregion` NHS region data (only England)
- `utla` Upper-tier local authority data (Again, only England)
- `ltla` Lower-tier local authority data (...only England)

## Data Structure
The `structure` parameter describes what metrics we want the data to describe. There are a lot of them, but the main metrics are `areaName`, `date` and `newCasesByPublishDate`. Click the arrow below to expand the full list of valid metrics.

<details class="govuk-details" data-module="govuk-details" closed=""><summary class="govuk-details__summary"><span class="govuk-details__summary-text">See a list of valid metrics for <code class="sc-fzoiQi ozSmQ">structure</code></span></summary><div class="govuk-details__text"><div class="govuk-!-margin-bottom-5"></div><dl><dt><code class="sc-fzoiQi ozSmQ">areaType</code></dt><dd>Area type as string</dd><dt><code class="sc-fzoiQi ozSmQ">areaName</code></dt><dd>Area name as string</dd><dt><code class="sc-fzoiQi ozSmQ">areaCode</code></dt><dd>Area Code as string</dd><dt><code class="sc-fzoiQi ozSmQ">date</code></dt><dd>Date as string [<code class="sc-fzoiQi ozSmQ">YYYY-MM-DD</code>]</dd><dt><code class="sc-fzoiQi ozSmQ">hash</code></dt><dd>Unique ID as string</dd><dt><code class="sc-fzoiQi ozSmQ">newCasesByPublishDate</code></dt><dd>New cases by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">cumCasesByPublishDate</code></dt><dd>Cumulative cases by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">cumCasesBySpecimenDateRate</code></dt><dd>Rate of cumulative cases by publish date per 100k resident population</dd><dt><code class="sc-fzoiQi ozSmQ">newCasesBySpecimenDate</code></dt><dd>New cases by specimen date</dd><dt><code class="sc-fzoiQi ozSmQ">cumCasesBySpecimenDateRate</code></dt><dd>Rate of cumulative cases by specimen date per 100k resident population</dd><dt><code class="sc-fzoiQi ozSmQ">cumCasesBySpecimenDate</code></dt><dd>Cumulative cases by specimen date</dd><dt><code class="sc-fzoiQi ozSmQ">maleCases</code></dt><dd>Male cases (by age)</dd><dt><code class="sc-fzoiQi ozSmQ">femaleCases</code></dt><dd>Female cases (by age)</dd><dt><code class="sc-fzoiQi ozSmQ">newPillarOneTestsByPublishDate</code></dt><dd>New pillar one tests by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">cumPillarOneTestsByPublishDate</code></dt><dd>Cumulative pillar one tests by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">newPillarTwoTestsByPublishDate</code></dt><dd>New pillar two tests by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">cumPillarTwoTestsByPublishDate</code></dt><dd>Cumulative pillar two tests by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">newPillarThreeTestsByPublishDate</code></dt><dd>New pillar three tests by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">cumPillarThreeTestsByPublishDate</code></dt><dd>Cumulative pillar three tests by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">newPillarFourTestsByPublishDate</code></dt><dd>New pillar four tests by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">cumPillarFourTestsByPublishDate</code></dt><dd>Cumulative pillar four tests by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">newAdmissions</code></dt><dd>New admissions</dd><dt><code class="sc-fzoiQi ozSmQ">cumAdmissions</code></dt><dd>Cumulative number of admissions</dd><dt><code class="sc-fzoiQi ozSmQ">cumAdmissionsByAge</code></dt><dd>Cumulative admissions by age</dd><dt><code class="sc-fzoiQi ozSmQ">cumTestsByPublishDate</code></dt><dd>Cumulative tests by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">newTestsByPublishDate</code></dt><dd>New tests by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">covidOccupiedMVBeds</code></dt><dd>COVID-19 occupied beds with mechanical ventilators</dd><dt><code class="sc-fzoiQi ozSmQ">hospitalCases</code></dt><dd>Hospital cases</dd><dt><code class="sc-fzoiQi ozSmQ">plannedCapacityByPublishDate</code></dt><dd>Planned capacity by publish date</dd><dt><code class="sc-fzoiQi ozSmQ">newDeaths28DaysByPublishDate</code></dt><dd>Deaths within 28 days of positive test</dd><dt><code class="sc-fzoiQi ozSmQ">cumDeaths28DaysByPublishDate</code></dt><dd>Cumulative deaths within 28 days of positive test</dd><dt><code class="sc-fzoiQi ozSmQ">cumDeaths28DaysByPublishDateRate</code></dt><dd>Rate of cumulative deaths within 28 days of positive test per 100k resident population</dd><dt><code class="sc-fzoiQi ozSmQ">newDeaths28DaysByDeathDate</code></dt><dd>Deaths within 28 days of positive test by death date</dd><dt><code class="sc-fzoiQi ozSmQ">cumDeaths28DaysByDeathDate</code></dt><dd>Cumulative deaths within 28 days of positive test by death date</dd><dt><code class="sc-fzoiQi ozSmQ">cumDeaths28DaysByDeathDateRate</code></dt><dd>Rate of cumulative deaths within 28 days of positive test by death date per 100k resident population</dd></dl></div></details>

We will look at new cases by publish date and new deaths by death date, so the structure will look like this

In [5]:
structure_cases_death = {
    "date": "date",
    "areaName": "areaName",
    "newCases": "newCasesByPublishDate",
    "cumCases": "cumCasesBySpecimenDate",
    "cumCasesRate": "cumCasesBySpecimenDateRate",
    "newDeaths": "newDeathsByDeathDate"
}

## Pulling and cleaning data

Now we create the class and get the DataFrame from it. We also use `fillna(0)` to fill any entries that are `NaN`'s - because that is the default if a value is missing.

In [6]:
uk_cases = Cov19API(filters=filter_all_nations,
                    structure=structure_cases_death).get_dataframe().fillna(0)

uk_cases['date'] = pd.to_datetime(uk_cases['date'], format='%Y-%m-%d')
uk_cases.sort_values(['areaName', 'date'], inplace=True)
uk_cases.reset_index(drop=True, inplace=True)

Note that the Welsh Government announced that 11,000 cases were missing from between the 9th and 15th of December. This explains the large spike after the 17th of December, and also the decrease in cases before that. See this [BBC article](https://www.bbc.com/news/uk-wales-55105307) and relevant announcement by [Public Health Wales](https://phw.nhs.wales/news/forthcoming-changes-to-the-way-we-publish-coronavirus-information/) about how they are changing the way they report cases.

In the data from the COVID19 API, all 11,000 cases are allocated to the 17th of December. To overcome this, we will evenly distribute the cases out over the preceeding 5 days. This may not be the most accurate way of doing it, but it will result in the cleanest picture when it comes to plotting the graphs.

In [7]:
date_list = ['2020-12-13', '2020-12-14',
             '2020-12-15', '2020-12-16', '2020-12-17']

uk_cases.iloc[(uk_cases.query("areaName=='Wales'").query("date==@date_list").index), 2] = np.flip(
    np.array(list(range(2494 + int((2801 - 2494)/6), 2801 - int((2801 - 2494)/6), int((2801 - 2494)/6)))))

Finally we add a column to the dataframe called `dailyChange` which will keep track of if the number of new cases has gone up or down per day.

In [8]:
grouped_df = uk_cases.groupby('areaName')

uk_cases['casesChange'] = grouped_df.apply(
    lambda x: x['newCases'] - x['newCases'].shift(1).fillna(0)).reset_index(drop=True)

uk_cases.sample(5, random_state=40)  # a random sample of rows

,date,areaName,newCases,cumCases,cumCasesRate,newDeaths,casesChange
482,2021-05-26,England,2540.0,3907262.0,6909.4,0,459.0
517,2021-06-30,England,21293.0,4218004.0,7458.9,0,4491.0
1876,2020-07-22,Wales,22.0,17095.0,539.3,0,0.0
1527,2021-02-28,Scotland,572.0,202038.0,3696.3,0,47.0
80,2020-04-19,England,3607.0,105413.0,186.4,0,59.0


Notice that only deaths in England have been counted in the newDeaths column. I prefer to look at the number of cases per 100k population, but to do this with the `newCases` column, we would need to grab population data for each country. Alternatively we can estimate the population by using the cumulative cases per 100k column - the cases per 100k is given by

$$
\text{cases per 100k} = 100000 * \frac{\text{cases}}{\text{population}}
$$

We will take numbers from the latest available day (just to make sure there are no zeros). For Wales:

In [9]:
wales_pop = round(100000 * uk_cases.query("areaName == 'Wales'").cumCases.max() /
                  uk_cases.query("areaName == 'Wales'").cumCasesRate.max())
print(f'Wales population: {wales_pop}')

Wales population: 3169579


which is about right (it was 3,152,879 in 2019..). And for the rest of the countries:

In [10]:
countries = ['Wales', 'Scotland', 'Northern Ireland', 'England']
countries_population = dict()
for country in countries:
    countries_population[country] = round(100000 * uk_cases.query(
        "areaName == @country").cumCases.max() / uk_cases.query("areaName == @country").cumCasesRate.max())

if 'population' not in uk_cases.columns:
    countries_pop_df = pd.DataFrame.from_dict(countries_population, orient='index', columns=[
        'population'])
    uk_cases = uk_cases.join(countries_pop_df, on='areaName')

uk_cases['newCasesRate'] = 100000 * uk_cases.newCases / uk_cases.population
uk_cases['casesChangeRate'] = 100000 * \
    uk_cases.casesChange / uk_cases.population

After all that, we just added some new columns that use the 'per 100k' metric. The last thing we will add is a column showing the number of cases over a 7 day period.

## Weekly cases

We will take the 7 day rolling sum of the new cases rate (i.e, new cases per 100k population) grouped by each country, and fill the missing values with 0's. 

In [11]:
uk_cases['weeklyCasesRate'] = uk_cases.groupby(by='areaName')['newCasesRate'].rolling(7).sum().reset_index(drop=True).fillna(0)

## Overview of UK cases

For the plotting we will also take the total cases for the UK. We could do this by grouping by date in the `uk_cases` dataframe and summing up the new cases like that - however we will just run another query with the `Cov19API` and run the same preprocessing as above, but this time cleaned up into a function.

In [12]:
overview_cases = Cov19API(filters=filter_all_uk, structure=structure_cases_death).get_dataframe().fillna(0)
def preprocess_dataframe(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df.sort_values('date', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['casesChange'] = df['newCases'] - df['newCases'].shift(-1).fillna(0)
    population = round(100000 * df.cumCases.max() /
                  df.cumCasesRate.max())
    df['newCasesRate'] = 100000 * df.newCases / population
    df['casesChangeRate'] = 100000 * df.casesChange / population
    df['weeklyCasesRate'] = df['newCasesRate'].rolling(7).sum().fillna(0)
    return df
preprocess_dataframe(overview_cases)

,date,areaName,newCases,cumCases,cumCasesRate,newDeaths,casesChange,newCasesRate,casesChangeRate,weeklyCasesRate
0,2020-01-30,United Kingdom,0.0,2.0,0.0,0,-2.0,0.000000,-0.002981,0.000000
1,2020-01-31,United Kingdom,2.0,2.0,0.0,0,2.0,0.002981,0.002981,0.000000
2,2020-02-01,United Kingdom,0.0,2.0,0.0,0,0.0,0.000000,0.000000,0.000000
3,2020-02-02,United Kingdom,0.0,2.0,0.0,0,0.0,0.000000,0.000000,0.000000
4,2020-02-03,United Kingdom,0.0,2.0,0.0,0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
593,2021-09-14,United Kingdom,26628.0,7344755.0,10949.0,0,-3969.0,39.695233,-5.916719,344.035791
594,2021-09-15,United Kingdom,30597.0,7372079.0,10989.8,0,3686.0,45.611952,5.494841,331.546432
595,2021-09-16,United Kingdom,26911.0,0.0,0.0,0,-5740.0,40.117111,-8.556806,314.996316
596,2021-09-17,United Kingdom,32651.0,0.0,0.0,0,2507.0,48.673917,3.737267,307.585883


# Plotting the data

We will use the Python library `Altair` for visualising the data, see the [altair docs](https://altair-viz.github.io/) for more information.

First we have a graph which shows the daily change in the number of new cases for each country. This number jumps up and down all over the place, which is likely due to delay in reporting of new cases over the weekend. Another interesting thing is that it looks like the daily cases in Wales experienced a much shorter period of calm over the summer (calm in the sense of daily cases not jumping up and down).

The orange bars are days when the number of new cases (per 100k population) was more than the previous day, while the blue are days when the number of new cases dropped. The red line is the 7 day moving average. 

When the moving average line is below 0, it means that there is a consistent drop in new cases. We can see this clearly happening around the times that lockdowns were introduced (though, to varying degrees). I will update the graphs soon with a marker of when each lockdown started.

In [13]:
# collapse
import altair as alt

bars = alt.Chart(uk_cases).mark_bar().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("casesChangeRate:Q", axis=alt.Axis(title='Change in daily cases per 100k')),
    tooltip='casesChange',
    color=alt.condition(
        alt.datum.casesChangeRate > 0,
        alt.value("orange"),  # The positive color
        alt.value("blue")  # The negative color
    )
).properties(title='Daily change in number of new cases with 7 day rolling mean',width=700).interactive()

line = alt.Chart(uk_cases).mark_line(
    color='red',
    size=2,
    opacity=0.6
).transform_window(
    rolling_mean='mean(casesChangeRate)',
    frame=[0, 7],
    groupby=['areaName']
).encode(
    x='yearmonthdate(date):T',
    y='rolling_mean:Q'
)

alt.layer(bars, line, data=uk_cases).facet(alt.Column(
    'areaName', title=''), columns=1).resolve_scale(y='independent').properties(title='Daily change in number of new cases in each country with 7 day rolling mean line')

alt.FacetChart(...)

Next is a bar chart of the number of new cases in each country (per 100k population), with the 7 day moving average of cases. Again we see that Wales saw a longer period of raising and falling cases compared to the other countries.

After a period of cases falling, each nation is now seeing a rise in the number of cases - especially in Wales.

In [14]:
# collapse
bars = alt.Chart(uk_cases).mark_bar().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("newCasesRate:Q", axis=alt.Axis(title='Daily new cases per 100k')),
    tooltip='newCasesRate',
    color=alt.condition(
        alt.datum.dailyChange > 0,
        alt.value("orange"),  # The positive color
        alt.value("blue")  # The negative color
    )
).properties(title='New cases per 100k population with rolling 7 day average', width=600).interactive()
line = alt.Chart(uk_cases).mark_line(
    color='red',
    size=2,
).transform_window(
    rolling_mean='mean(newCasesRate)',
    frame=[0, 7],
    groupby=['areaName']
).encode(
    x='yearmonthdate(date):T',
    y='rolling_mean:Q'
)
alt.layer(line, bars, data=uk_cases).facet(alt.Row('areaName', title='Country'), columns=1)

alt.FacetChart(...)

In [15]:
#collapse
countries = uk_cases['areaName'].unique()
countries.sort()

selection = alt.selection_single(
    name='Select',
    fields=['areaName'],
    init={'areaName': 'Wales'},
    bind={'areaName': alt.binding_select(options=countries)}
)

# scatter plot, modify opacity based on selection
bars = alt.Chart(uk_cases).mark_bar().add_selection(
    selection
).encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("weeklyCasesRate:Q", axis=alt.Axis(title='Incidence rate')),
    tooltip='weeklyCasesRate:Q',
    opacity=alt.condition(selection, alt.value(1), alt.value(0))
).properties(title=f'7 day incidence rate of individual countries vs rolling mean across the UK', width=600)

line = alt.Chart(overview_cases).mark_line(
    color='red',
    size=2,
).transform_window(
    rolling_mean='mean(weeklyCasesRate)',
    frame=[0, 7]
).encode(
    x='yearmonthdate(date):T',
    y='rolling_mean:Q'
)

alt.layer(bars, line)

alt.LayerChart(...)